# manuela CW measurements

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
from scipy import constants as const

def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

allSpectra = np.genfromtxt(r"C:\Users\M\Documents\Books\masterprojectinformation\images\2021-07-05ManuelaSQIB.txt", skip_header=2)
percent = [10, 4, 2, 1, 0.5]
ind_percent = [1,2,3,4,5]
wavelengths = allSpectra[:,0]
boolArr = wavelengths < 1000
wavelengths = wavelengths[boolArr]
relTransmissionMain = allSpectra[boolArr,4]

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,3), dpi = 288)
for ind in [0,1,3]:
    ax.plot(wavelengths, allSpectra[boolArr,ind+1]*100, label = (r"%.1f wt" %percent[ind] + r"\%"))

secax = ax.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E \ / \ eV}$")

ax.set_yscale('log')
ax.set_ylabel('transmission / \%')
ax.set_xlabel('wavelength / nm')
ax.legend(loc = "lower right")
plt.show()

### SQIB (manuela) vs Anthracene

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

allSpectra = np.genfromtxt(r"C:\Users\M\Documents\Books\masterprojectinformation\images\2021-07-05ManuelaSQIB.txt", skip_header=2, usecols=(0,4))

wavelengths = allSpectra[:,0]
boolArr = wavelengths < 1000
wavelengths = wavelengths[boolArr]
relTransmission = allSpectra[boolArr,1]

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,3), dpi = 288)
ax.plot(wavelengths, relTransmission*100)
ax.set_yscale('log')
ax.set_ylabel('transmission / \%')
ax.set_xlabel('wavelength / nm')
plt.show()

### Red LEDs

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

allSpectra = np.genfromtxt(r"C:\Users\M\Documents\Books\masterprojectinformation\images\SpectrumRedLEDsforCalibration.txt", skip_header=17, skip_footer=1)

wavelengths = allSpectra[:,0]
boolArr = (wavelengths > 500) & (wavelengths < 800)
emission = allSpectra[:,1]

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(2,2.5), dpi = 288)
ax.plot(wavelengths[boolArr], emission[boolArr]/np.max(emission))
ax.set_ylabel('emission / a.u.')
ax.set_xlabel('wavelength / nm')
ax.set_xticks([500,600,653,700,800],[500,600,653,700,800])

secax = ax.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([653])), 2))
secax.set_xlabel(r"$\mathrm{E \ / \ eV}$")

plt.show()